In [0]:
%run ../config/init_config


In [0]:
from pyspark.sql.functions import from_json, to_date, lit, col, row_number, broadcast, count, sum
from pyspark.sql.window import Window

In [0]:
def process_orders_silver():
    orders_schema = "order_id STRING, customer_id STRING, order_date STRING"

    query = (spark.readStream
                .format("delta")
                .table("bronze_raw")
                .filter("source_type = 'orders'")
                .select(from_json("value", orders_schema).alias("v"), "processed_time")
                .select("v.*", "processed_time")
                .withColumn("order_date", to_date("order_date", "yyyy-mm-dd"))
                .writeStream
                .format("delta")
                .option("mergeSchema", "true")
                .option("checkpointLocation", silver_checkpoint_path + "/orders")
                .trigger(availableNow=True)
                .table("order_silver")
                )
                        

    query.awaitTermination()

In [0]:
process_orders_silver()